In [1]:
from valuation.Manager import Valuation
from utilities.utils import fullpath
import numpy as np
import pandas as pd

In [2]:
value = Valuation(20231231)

In [3]:
df_weights = value.calculate_num_shares_on_rebalance_day()

In [7]:
df_weights = df_weights.loc[df_weights["Symbol"] != "cash", ["Symbol", "NumShares"]]

In [8]:
prices_path = fullpath("data", "market","prices","20231231.csv")
prices_path

'data/market/prices/20231231.csv'

In [9]:
df_prices = pd.read_csv(prices_path)
df_prices

,Date,ACA.PA,AI.PA,AIR.PA,ALO.PA,BN.PA,BNP.PA,CA.PA,CAP.PA,CS.PA,...,SAN.PA,SGO.PA,STLAM.MI,STMPA.PA,SU.PA,TEP.PA,TTE.PA,UBLB.F,VIE.PA,VIV.PA
0,2020-12-31,7.492945,103.125793,86.308441,45.444176,46.818352,33.533649,12.160026,119.008942,15.259549,...,67.429283,33.412476,NaN,29.275112,109.498657,247.890503,26.913239,NaN,16.549694,9.328155
1,2021-01-04,7.420339,104.815750,86.414185,47.043152,47.375713,33.459743,12.558716,120.463699,15.201676,...,67.686325,34.187641,9.458270,30.686657,112.691978,256.205322,26.870504,NaN,16.648941,9.310473
2,2021-01-05,7.445750,103.701912,86.356506,46.311913,47.549889,33.389725,13.009409,118.821220,15.195420,...,67.677757,34.098549,9.432844,30.957367,111.627533,250.448914,27.577522,NaN,16.557964,9.215000
3,2021-01-06,7.747066,103.932365,86.683357,47.394154,47.967907,35.233475,13.295425,117.366470,15.844529,...,67.266487,35.639977,9.234526,31.005705,114.728302,240.854950,28.789549,NaN,17.079018,9.359979
4,2021-01-07,7.805151,104.738937,86.154625,46.370415,47.218952,36.026993,13.364762,115.958626,15.942286,...,66.589630,37.849651,9.399792,31.324759,117.088585,244.509796,29.104204,NaN,17.343681,9.253898
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
766,2023-12-21,11.953912,157.903107,136.995956,12.215000,56.240971,58.912262,15.672743,186.039841,27.815157,...,85.414047,64.367645,19.885952,44.840515,177.782562,131.231308,57.387878,3.155079,27.868332,9.332608
767,2023-12-22,12.004350,158.457016,137.905823,12.005000,56.260262,59.034058,15.734280,185.400345,27.923496,...,85.778900,64.873932,19.867231,44.830666,178.294907,128.626938,57.665569,3.155079,27.849165,9.361846
768,2023-12-27,12.062260,157.974579,137.886047,12.205000,56.347084,59.015324,15.734280,188.007477,27.866972,...,85.615677,64.942085,19.871910,44.983337,179.221054,128.385788,57.822922,3.155079,27.523333,9.428117
769,2023-12-28,12.009953,157.420670,138.143173,12.135000,56.443550,58.509396,15.686943,186.728516,27.725658,...,85.702095,64.640259,19.829790,44.825737,178.689011,129.350372,56.841770,3.174317,27.456249,9.412524


In [10]:
prices = df_prices.fillna(0).set_index("Date").values
prices

array([[  7.49294472, 103.12579346,  86.30844116, ...,   0.        ,
         16.54969406,   9.32815456],
       [  7.42033863, 104.81575012,  86.41418457, ...,   0.        ,
         16.64894104,   9.31047344],
       [  7.44575024, 103.70191193,  86.35650635, ...,   0.        ,
         16.55796432,   9.21500015],
       ...,
       [ 12.06225967, 157.97457886, 137.88604736, ...,   3.15507865,
         27.5233326 ,   9.4281168 ],
       [ 12.0099535 , 157.42066956, 138.14317322, ...,   3.17431688,
         27.45624924,   9.41252422],
       [ 12.00434971, 157.34918213, 138.24208069, ...,   3.11660218,
         27.36999893,   9.43006611]], shape=(771, 40))

In [11]:
np.matrix(prices).shape

(771, 40)

In [12]:
numShares = np.matrix(df_weights.loc[:, "NumShares"].values)

In [13]:
numShares.transpose().shape

(40, 1)

In [14]:
ptf_value = prices @ numShares.transpose()

In [19]:
df_ptf = pd.DataFrame(ptf_value, columns=["ptf_value"])
df_ptf

,ptf_value
0,6.433082e+08
1,6.542299e+08
2,6.507381e+08
3,6.574506e+08
4,6.622519e+08
...,...
766,1.005882e+09
767,1.003655e+09
768,1.003235e+09
769,9.987393e+08


In [20]:
df_ptf["ptf_value"]

0      6.433082e+08
1      6.542299e+08
2      6.507381e+08
3      6.574506e+08
4      6.622519e+08
           ...     
766    1.005882e+09
767    1.003655e+09
768    1.003235e+09
769    9.987393e+08
770    9.999963e+08
Name: ptf_value, Length: 771, dtype: float64

In [3]:
df_prices

NameError: name 'df_prices' is not defined

In [ ]:
from decimal import getcontext, Decimal
import pandas as pd
import numpy as np
from utilities.utils import fullpath, last_working_day
from datetime import datetime

In [ ]:
weights_path = fullpath("data","strategy","caps","20231231.csv")
weights_path

In [ ]:
price_path = fullpath("data", "market", "prices","20231231.csv")
price_path

In [ ]:
ptf_size = 1e9
ptf_size

In [ ]:
weights = pd.read_csv(weights_path)
weights

In [ ]:
prices = pd.read_csv(price_path)
prices['Date'] = pd.to_datetime(prices['Date'])
prices['Date'] = prices['Date'].dt.date
prices.tail()

In [ ]:
px = prices[prices.Date == last_working_day("20231231")].drop("Date", axis=1).transpose()
px = px.reset_index()
px.columns = ["Symbol", "Prices"]
px



In [ ]:
df_prices_weights = pd.merge(px, weights, left_on="Symbol", right_on="Symbol")
df_prices_weights.head()

In [ ]:
df_prices_weights["DollarWeight"] = df_prices_weights["Weights"].apply(lambda x: ptf_size*x )
df_prices_weights["NumShares"] = df_prices_weights["DollarWeight"].values / df_prices_weights["Prices"].values
df_prices_weights

In [ ]:
df_prices_weights["NumShares"] = np.floor( df_prices_weights["DollarWeight"] / df_prices_weights["Prices"] )
cash = 1e9 - sum(df_prices_weights["NumShares"]*df_prices_weights["Prices"])
cash

In [ ]:
df_prices_weights[["Symbol","Weights","NumShares","DollarWeight"]]

In [ ]:
d = {"Symbol": "cash", "Prices": 1, "Weights": cash/ptf_size, "DollarWeight": cash, "NumShares": cash }


In [ ]:
df_cash = pd.DataFrame({"Symbol": "cash", "Prices": 1, "Weights": cash/ptf_size, "DollarWeight": cash, "NumShares": cash }, index=range(1))
df_cash

In [ ]:
df_asset_table = pd.concat([df_prices_weights, df_cash])
df_asset_table

In [ ]:
weights = pd.read_csv(weights_path)
weights.head()

In [ ]:
price_path = fullpath("data", "market", "prices","20231231.csv")
price_path
prices = pd.read_csv(price_path)
prices['Date'] = pd.to_datetime(prices['Date'])
prices['Date'] = prices['Date'].dt.date
prices.set_index("Date", inplace=True)
prices.tail()


In [ ]:
last_working_day("20231231")

In [ ]:
prices[prices.index == last_working_day("20231231")]


In [ ]:
px = prices[prices["Date"] == last_working_day("20231231")]
px

In [ ]:
px.drop("Date", axis=1, inplace=True)


In [ ]:
px = px.transpose()

In [ ]:
pd.merge(px, weights, left_index=True, right_on="Symbol")

In [ ]:
px1 = px.reset_index()

In [ ]:
px1

In [ ]:
px1

In [ ]:
weights

In [ ]:
pd.merge(px1, weights)

In [ ]:
pd.merge(px, weights, how="inner", left_on="Symbol", right_on="Symbol")

In [ ]:
px1

In [ ]:
weights

In [ ]:
# Ensure 'Symbol' is the index for both DataFrames
weights = weights.set_index("Symbol")
px1 = px1.set_index("Symbol")

# Merge the two DataFrames on the 'Symbol' index
merged_df = weights.merge(px1, left_index=True, right_index=True, how="inner")

# Display the merged DataFrame
merged_df

#pd.merge(weights, px1, left_on="Symbol", right_on="Symbol")

In [ ]:
# t = prices.Date.apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d").date())

In [ ]:
# # Ensure 'Date' column is in datetime format
# prices['Date'] = pd.to_datetime(prices['Date'])

# # Create a date range from the first date in prices to 2023-12-31
# date_range = pd.date_range(start=prices['Date'].min(), end='2023-12-31', freq='D')

# # Create a DataFrame with the date range
# dates_df = pd.DataFrame({'Date': date_range})

# # Merge the dates DataFrame with the prices DataFrame
# prices = pd.merge(dates_df, prices, on='Date', how='left')

# # Forward-fill missing data
# prices = prices.ffill()

# # Display the updated DataFrame
# prices.tail()

In [ ]:
# date = datetime(2023,12,30) # pd.to_datetime("2023-12-01",format="%Y-%m-%d")
# last_day_of_month = date + pd.offsets.MonthEnd(0)
# last_day_of_month

# if last_day_of_month.weekday() in [5, 6]:  # 5 = Saturday, 6 = Sunday
#     # If it's a weekend, subtract days to get the last working day
#     last_working_day = last_day_of_month - pd.offsets.Week(weekday=4)  # 4 = Friday
# else:
#     last_working_day = last_day_of_month

# last_working_day.date()

In [ ]:
weights["Dollar"] = weights["Weights"].apply(lambda x: x*ptf_size)
weights

In [ ]:
last_working_day("20231231")

In [ ]:
idx = prices["Date"] == last_working_day("20231231")

In [ ]:
prices.loc[idx, :]

In [ ]:
weights.set_index("Symbol").transpose()

In [ ]:
pd.merge(weights, prices, how=  )

In [ ]:
?pd.merge